In [1]:
import pandas as pd

In [2]:
# Using the example from the lecture notes
txn_dict = {'10': 'apple,carrot,durian', '20': 'banana,carrot,edamame',
            '30': 'apple,banana,carrot,edamame', '40': 'banana,edamame'}


trans_df = pd.DataFrame.from_dict(txn_dict,orient='index')

trans_df = trans_df[0].str.split(',')

display(trans_df)

10             [apple, carrot, durian]
20           [banana, carrot, edamame]
30    [apple, banana, carrot, edamame]
40                   [banana, edamame]
Name: 0, dtype: object

In [3]:
from apriori_algorithm import AprioriAlgorithm

apriori = AprioriAlgorithm(minsup=2, verbose=True)

In [4]:
freq_item_sets = apriori.run(trans_df)

Dataset size: 5
Minsup = 2 

Freq itemset table (Pruned):
   item_sets  supp_count
0     apple           2
3    banana           3
1    carrot           3
4   edamame           3

Candidate itemset table (Counting):
            item_sets  supp_count
0    (apple, banana)           1
1    (apple, carrot)           2
2   (apple, edamame)           1
3   (banana, carrot)           2
4  (banana, edamame)           3
5  (carrot, edamame)           2
Minsup = 2 

Freq itemset table (Pruned):
            item_sets  supp_count
1    (apple, carrot)           2
3   (banana, carrot)           2
4  (banana, edamame)           3
5  (carrot, edamame)           2

Candidate itemset table (Counting):
                    item_sets  supp_count
0  (banana, carrot, edamame)           2
Minsup = 2 

Freq itemset table (Pruned):
                    item_sets  supp_count
0  (banana, carrot, edamame)           2


/Users/siddhantpathak/Desktop/Projects/FrequentMining/apriori_algorithm.py:26: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if len(df) > 1 or (len(df) == 1 and int(df.supp_count >= self.supp)):


In [6]:
display(freq_item_sets)

,item_sets,supp_count
0,"(banana, carrot, edamame)",2
